We use llama_cpp to evaluate LLM followed by this article:

https://towardsdatascience.com/gguf-quantization-with-imatrix-and-k-quantization-to-run-llms-on-your-cpu-02356b531926

Only use the evaluation part.

# Install llama cpp

In [2]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!export GGML_CUDA=1 
!make > make.log 2>&1

Cloning into 'llama.cpp'...
remote: Enumerating objects: 34581, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 34581 (delta 46), reused 71 (delta 36), pack-reused 34470 (from 1)
Receiving objects: 100% (34581/34581), 57.57 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (24992/24992), done.
/kaggle/working/llama.cpp


In [3]:
!tail make.log

c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -c examples/cvector-generator/cvector-generator.cpp -o examples/cvector-generator/cvector-generator.o
c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  ggml/src/llamafile/sgemm.o ggml/src/ggml.o ggml/src/ggml-alloc.o ggml/src/ggml-backend.o ggml/src/ggml-quants.o ggml/src/ggml-aarch64.o src/llama.o src/llama-vocab.o src/ll

In [4]:
!pip install --force-reinstall -r requirements.txt > pip_install.log 2>&1
!tail pip_install.log

pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
rmm 24.8.2 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
s3fs 2024.6.1 requires fsspec==2024.6.1.*, but you have fsspec 2024.9.0 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow-metadata 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.25.5 which is incompatible.
tensorflow-transform 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.25.5 which is incompatible.
ydata-profiling 4.9.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.0 which is incompatible.


# Download gguf model

## Prepare huggingface token

In [5]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()

os.environ["HF_TOKEN"]=user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(os.environ["HF_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Download gguf model

In [6]:
from huggingface_hub import snapshot_download
model_name = "google/gemma-2-2b-it" # the model we want to quantize
methods = ['Q4_K_S','Q4_K_M'] #the methods to be used for quantization
base_model = "./original_model_gemma2-2b/" # where the FP16 GGUF model will be stored
quantized_path = "./quantized_model_gemma2-2b/" #where the quantized GGUF model will be stored
original_model = quantized_path + 'FP16.gguf'

snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1204: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/29.1k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

'/kaggle/working/llama.cpp/original_model_gemma2-2b'

## Convert model to gguf 

In [7]:
!mkdir -p /kaggle/working/llama.cpp/quantized_model_gemma2-2b/
!python convert_hf_to_gguf.py "/kaggle/working/llama.cpp/original_model_gemma2-2b/" --outfile "/kaggle/working/llama.cpp/quantized_model_gemma2-2b/FP16.gguf"

Writing: 100%|███████████████████████████| 5.23G/5.23G [00:22<00:00, 237Mbyte/s]


## Get wiki text as dataset

In [8]:
!wget https://object.pouta.csc.fi/OPUS-Wikipedia/v1.0/mono/en.txt.gz
!gunzip en.txt.gz
!head -n 10000 en.txt > en-h10000.txt
!sh scripts/get-wikitext-2.sh

--2024-09-23 16:24:21--  https://object.pouta.csc.fi/OPUS-Wikipedia/v1.0/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 532958396 (508M) [application/gzip]
Saving to: 'en.txt.gz'

en.txt.gz           100%[===================>] 508.27M  18.6MB/s    in 27s     

2024-09-23 16:24:49 (18.7 MB/s) - 'en.txt.gz' saved [532958396/532958396]

--2024-09-23 16:25:05--  https://huggingface.co/datasets/ggml-org/ci/resolve/main/wikitext-2-raw-v1.zip
Resolving huggingface.co (huggingface.co)... 3.165.160.11, 3.165.160.59, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.11|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/c6/78/c67802fcd48fa6f6a86773410b21cc6db1c5c546b20683b6c30b95f327a66922/ef7edb566e3e2b2d31b29c1fd

# Benchmarking the Perplexity 

Perplexity can be used to compare the models before and after quantization or other method. Here is a explaination of why we could not compare different models by benchmarking perplexity

https://thesalt.substack.com/p/why-cant-we-compare-the-perplexity


In [9]:
!./llama-perplexity -m /kaggle/working/llama.cpp/quantized_model_gemma2-2b/FP16.gguf -f wikitext-2-raw/wiki.test.raw --chunks 16

build: 3810 (1d48e98e) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
llama_model_loader: loaded meta data with 38 key-value pairs and 288 tensors from /kaggle/working/llama.cpp/quantized_model_gemma2-2b/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model_Gemma2 2b
llama_model_loader: - kv   3:                           general.basename str              = original_model_gemma2
llama_model_loader: - kv   4:                         general.size_label str              = 2B
llama_model_loader: - kv   5:                            general.license str              = gemma
llama_model_loader: 

# Benchmarking the Inference Throughput and Memory Consumption 

In [13]:
!./llama-bench -m /kaggle/working/llama.cpp/quantized_model_gemma2-2b/FP16.gguf -n 16 -mg 1

| model                          |       size |     params | backend    | threads |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | ------: | ---------: | ------------: | -------------------: |
| gemma2 2B F16                  |   5.97 GiB |     3.20 B | CPU        |       2 |          1 |         pp512 |         20.42 ± 0.16 |
| gemma2 2B F16                  |   5.97 GiB |     3.20 B | CPU        |       2 |          1 |          tg16 |          3.68 ± 0.13 |

build: 1d48e98e (3810)
